In [1]:
pip --version

pip 23.2.1 from C:\Users\KAJAL\anaconda3\Lib\site-packages\pip (python 3.11)

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [4]:
data={"intents":[
       {"tag":"greeting",
        "patterns":["Hello","How are you?","Hi There","Hi","What's up"],
        "responses":["Howdy Partner!","Hello","How are you doing?","Greetings!","How do you do"]
       },
       {"tag":"age",
        "patterns":["How old you are","when is your birthday","when was you born"],
        "responses":["I am 21 years old","I was born in 2003","My birthday is June 2nd and I was born in 2003","02/06/2003"]
       },
       {"tag":"date",
        "patterns":["what are you doing this weekend",
                    "do you want to hangout sometime?","what are your plans for this week"],
        "responses":["I am available this week","I don't have any Plans","I am  not busy"]
       },
       {"tag":"name",
        "patterns":["what's your name","what are you called","who are you"],
        "responses":["my name is kajal","I'm kajal","kajal"]
       },
       {"tag":"goodbye",
        "patterns":["bye","g2g","see ya","adios","cya"],
        "responses":["It was nice speaking to you","See you later","Speak Soon","Greetings!","How do you do"]
       },
]}

In [5]:
import json
import string
import random

import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout

nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\KAJAL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\KAJAL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
lemmatizer=WordNetLemmatizer()

words=[]
classes=[]
doc_x=[]
doc_y=[]

In [7]:
for intent in data ["intents"]:
    for pattern in intent["patterns"]:
        tokens=nltk.word_tokenize(pattern)
        words.extend(tokens)
        doc_x.append(pattern)
        doc_y.append(intent["tag"])
    if intent["tag"]not in classes:
        classes.append(intent["tag"])

In [8]:
word=[lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]

In [9]:
words=sorted(set(words))
classes=sorted(set(classes))

In [10]:
print(words)

["'s", '?', 'Hello', 'Hi', 'How', 'There', 'What', 'adios', 'are', 'birthday', 'born', 'bye', 'called', 'cya', 'do', 'doing', 'for', 'g2g', 'hangout', 'is', 'name', 'old', 'plans', 'see', 'sometime', 'this', 'to', 'up', 'want', 'was', 'week', 'weekend', 'what', 'when', 'who', 'ya', 'you', 'your']


In [11]:
print(classes)

['age', 'date', 'goodbye', 'greeting', 'name']


In [12]:
print(doc_x)

['Hello', 'How are you?', 'Hi There', 'Hi', "What's up", 'How old you are', 'when is your birthday', 'when was you born', 'what are you doing this weekend', 'do you want to hangout sometime?', 'what are your plans for this week', "what's your name", 'what are you called', 'who are you', 'bye', 'g2g', 'see ya', 'adios', 'cya']


In [13]:
print(doc_y)

['greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'age', 'age', 'age', 'date', 'date', 'date', 'name', 'name', 'name', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye']


list for training data

In [15]:
training=[]
out_empty=[0]*len(classes)

#creating a bag of words model

for idx,doc in enumerate(doc_x):
    bow=[]
    text=lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    output_row=list(out_empty)
    output_row[classes.index(doc_y[idx])]=1
    
    training.append([bow, output_row])
    
random.shuffle(training)

training=np.array(training,dtype=object)

train_x=np.array(list(training[:,0]))
train_y=np.array(list(training[:,1]))

In [16]:
input_shape=(len(train_x[0]),)
output_shape=len(train_y[0])

epocha=500

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout

#create a sequential model
model=Sequential()
model.add(Dense(128, input_shape=input_shape,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(output_shape,activation='softmax'))

adam=tf.keras.optimizers.Adam(learning_rate=0.01)

model.compile(loss='categorical_crossentropy',
             optimizer=adam,
             metrics=['accuracy'])

print(model.summary())

C:\Users\KAJAL\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 128)                 │           4,992 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 5)                   │             325 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 13,573 (53.02 KB)

 Trainable params: 13,573 (53.02 KB)

 Non-trainable params: 0 (0.00 B)

None


In [18]:
model.fit(x=train_x,y=train_y,epochs=500,verbose=1)

Epoch 1/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.1579 - loss: 1.6137
Epoch 2/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.3684 - loss: 1.4756
Epoch 3/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.4211 - loss: 1.4118
Epoch 4/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.6316 - loss: 1.2694
Epoch 5/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.4737 - loss: 1.1694
Epoch 6/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.6316 - loss: 1.1189
Epoch 7/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.7368 - loss: 0.9586
Epoch 8/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.5789 - loss: 0.9423
Epoch 9/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.7368 - loss: 0.8288
Epoch 10/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.7895 - loss: 0.7452
Epoch 11/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.7368 - loss: 0.6886
Epoch 12/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.7895 - loss

In [19]:
def clean_text(text):
    tokens=nltk.word_tokenize(text)
    tokens=[lemmatizer.lemmatize(word)for word in tokens]
    return tokens

def bag_of_words(text,vocab):
    tokens=clean_text(text)
    bow=[0]*len(vocab)
    for w in tokens:
        for idx,word in enumerate(vocab):
            if word==w:
                bow[idx]=1
    return np.array(bow)

In [22]:
def pred_class(text,vocab,labels):
    bow=bag_of_words(text,vocab)
    result=model.predict(np.array([bow]))[0]
    thresh=0.2
    y_pred=[[idx,res] for idx ,res in enumerate(result) if res>thresh]
    
    y_pred.sort(key=lambda x:x[1],reverse=True)
    return_list=[]
    for r in y_pred:
        return_list.append(labels[r[0]])
    return return_list

def get_response(intents_list,intents_json):
    tag=intents_list[0]
    list_of_intents=intents_json["intents"]
    for i in list_of_intents:
        if i["tag"]==tag:
            result=random.choice(["responses"])
            break
    return result

In [ ]:
while True:
    message=input("")
    intents=pred_class(message,words,classes)
    result=get_response(intents,data)
    print(result)

hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
responses
what's your name
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
responses
goodbye
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
responses
